# Web2Post — From Web Search to Ready-to-Publish Social Posts

## 📌 Problem Statement

Creating high-quality social media posts around trending topics is a time-consuming process. It requires manually searching the web, filtering reliable information, understanding the context, summarizing key insights, and then rewriting the same content differently for each social media platform.

Each platform has its own tone, structure, and audience expectations:
- LinkedIn requires professional, thoughtful storytelling
- Twitter/X demands short, punchy, attention-grabbing posts
- Facebook favors casual, fun, caption-style content

This repetitive workflow slows down content creation and makes it difficult to respond quickly to trending topics.

**Web2Post** aims to automate this entire pipeline using LangChain by:
1. Searching the web for a given topic  
2. Generating a factual, noise-free summary as a single source of truth  
3. Converting that summary into platform-specific social media posts  

The result is a fast, consistent, and scalable way to transform web information into ready-to-publish social media content.

---


In [ ]:
!pip install langchain
!pip install langchain-google-genai
!pip install google-generativeai
!pip install tavily-python

In [2]:
from google.colab import userdata
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser,PydanticOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from tavily import TavilyClient
from pydantic import BaseModel, Field
from typing import Dict, List
from langchain_core.runnables import RunnableParallel
import json

In [5]:
GEMINI_API_KEY=userdata.get("GEMINI_API_KEY")
TAVILY_API_KEY=userdata.get("tavily")
client = TavilyClient(userdata.get("tavily"))
llm=ChatGoogleGenerativeAI(model="gemini-2.5-flash",google_api_key=GEMINI_API_KEY)

In [6]:
def web_search(topic):
  response=client.search(
      query=topic,
      include_answer="basic"
  )
  return response.get("answer")


In [14]:
summary_prompt = PromptTemplate(
    template="""
You are a professional research analyst.

Your task is to read the following web search content and produce a clear, factual summary.

Web Content:
{web_content}

Summarization Rules:
- Extract only verified, meaningful information.
- Remove repetition, ads, opinions, and filler content.
- Do NOT speculate or add new information.
- Keep the language simple, neutral, and easy to understand.
- Focus on what happened, why it matters, and any key facts or outcomes.
- The summary must be reliable enough to act as the single source of truth for writing social media posts.

Output Requirements:
- Write in plain text.
- Use short paragraphs or bullet points if helpful.
- Do NOT include hashtags, emojis, or promotional language.
- Do NOT mention sources or URLs.

Return only the final summary.
""",
    input_variables=["web_content"]
)


summary_chain= summary_prompt | llm | StrOutputParser()

In [11]:
linkedin_prompt = PromptTemplate(
    template="""
You are a professional LinkedIn content strategist.

Using the summary below, write a LinkedIn post.

Summary:
{summary}

Rules for LinkedIn:
- Write in first person.
- Maintain a professional, thoughtful, value-driven tone.
- Use 2 to 4 short paragraphs.
- Focus on insights, learnings, or implications.
- Avoid emojis.
- Avoid hashtags unless absolutely necessary (prefer zero hashtags).
- Do NOT sound promotional or clickbait.

Return only the LinkedIn post text.
""",
    input_variables=["summary"]
)

twitter_prompt = PromptTemplate(
    template="""
You are a social media copywriter for Twitter/X.

Using the summary below, write a single tweet.

Summary:
{summary}

Rules for Twitter/X:
- Must be under 280 characters.
- Punchy, scroll-stopping, and conversational.
- Use 1–2 relevant emojis.
- Include 2–3 relevant hashtags.
- Write in a casual, tweet-style tone.
- No threads, only ONE tweet.

Return only the tweet text.
""",
    input_variables=["summary"]
)

facebook_prompt = PromptTemplate(
    template="""
You are a creative Facebook content writer.

Using the summary below, write a Facebook caption.

Summary:
{summary}

Rules for Facebook:
- Short, catchy, and fun.
- Ideally under 150 characters.
- Use emojis naturally.
- Optional hashtags (0–2 max).
- Casual, friendly, caption-like tone.
- Not formal or professional sounding.

Return only the Facebook caption.
""",
    input_variables=["summary"]
)


In [12]:
twitter_chain= twitter_prompt | llm | StrOutputParser()
linkedin_chain= linkedin_prompt | llm | StrOutputParser()
facebook_chain= facebook_prompt | llm | StrOutputParser()

parallel_chain=RunnableParallel(
    {
        "twitter_post": twitter_chain,
        "linkedin_post": linkedin_chain,
        "facebook_caption": facebook_chain
    }
)


In [15]:
topic=input("Enter the topic name :")
web_content=web_search(topic)
summary=summary_chain.invoke({"web_content":web_content})
response=parallel_chain.invoke({"summary":summary})

Enter the topic name :messi in india


In [26]:
final_posts={
    "query":topic,
    "summary":summary,
    "linkedin_post":response["linkedin_post"],
    "twitter_post":response["twitter_post"],
    "facebook_caption":response["facebook_caption"]
}

with open("posts.json", "w") as f:
    json.dump(final_posts, f, indent=2)

print(json.dumps(final_posts, indent=2))

{
  "query": "messi in india",
  "summary": "Lionel Messi's 2025 promotional tour in India aimed to increase football's popularity.\n\nThe tour faced disruptions in Kolkata, where fans threw chairs and bottles.\n\nFollowing the incident in Kolkata, the tour continued uneventfully in Hyderabad, Mumbai, and Delhi.",
  "linkedin_post": "I've been reflecting on the recent promotional tour for Lionel Messi in India, which aimed to significantly increase football's popularity across the nation. The tour's initial stop in Kolkata faced notable disruptions, reminding us of the immense complexities involved in managing large-scale public events, particularly when enthusiasm for a global icon meets varied logistical realities.\n\nThis incident served as a powerful illustration of how critical detailed planning and crowd management strategies are, especially in diverse and highly passionate markets. Ensuring a safe and productive environment for both fans and participants is paramount to achievin